# This is a short notebook to show sample code for my master's application to MILA. 

## Thank you for taking the time to read this. Feel free to reachout if you have any questions or comments regarding this. 

#### Vincent Dandenault - vincent.dandenault@polymtl.ca 